## Char-rnn 笑傲江湖 - Continue training 

### Setups

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import random

In [2]:
use_gpu = torch.cuda.is_available()

In [4]:
f= 'data/xiaoaojianghu_jinyong.txt'
file = open(f, encoding='utf-16').read()

In [5]:
file_len = len(file)
n_characters = len(set(file))

In [7]:
def create_lookup_tables(file):
    vocab = set(file)
    vocab_to_int = {word: idx for idx, word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab

In [8]:
#vocab_to_int, int_to_vocab = create_lookup_tables(file)
import pickle
with open('vocab_to_int.pickle', 'rb') as handle:
    vocab_to_int = pickle.load(handle)
    
with open('int_to_vocab.pickle', 'rb') as handle:
    int_to_vocab = pickle.load(handle)

In [7]:
chunk_len = 300

def random_chunk(chunk_len = 100):
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk(100))

教的只是剑法，但于他议论风范，不但钦仰敬佩，更是觉得亲近之极，说不出的投机。风清扬是高了他两辈的太师叔，可是令狐冲内心，却隐隐然有一股平辈知己、相见恨晚的交谊，比之恩师岳不群，似乎反而亲切得多，心想：“


In [9]:
def char2tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = vocab_to_int[string[c]]
    return tensor

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="gru",n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.model = model
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self,batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


In [11]:
def random_training_set(chunk_len,batch_size):    
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for i in range(batch_size):
        chunk = random_chunk(chunk_len)
        inp[i] = char2tensor(chunk[:-1])
        target[i] = char2tensor(chunk[1:])
    if use_gpu:
        inp = inp.cuda()
        target = target.cuda()
    else:
        inp = Variable(inp)
        target = Variable(target)
    return inp, target

In [12]:
def evaluate(decoder, prime_str='A', predict_len=100, temperature=0.8, use_gpu = False):
    hidden = decoder.init_hidden(1)
    prime_input = char2tensor(prime_str).unsqueeze(0)
    
    if use_gpu:
        if isinstance(hidden, tuple):
            hidden = (hidden[0].cuda(), hidden[1].cuda())
        else:
            hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
    
    inp = prime_input[:,-1]
    if use_gpu:
        inp = inp.cuda()
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)      
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        i = top_i.data[0].item()
        #print (i)
        
        # Add predicted character to string and use as next input
        predicted_char = int_to_vocab[i]
        predicted += predicted_char
        inp = char2tensor(predicted_char).unsqueeze(0)
        if use_gpu:
            inp = inp.cuda()

    return predicted

In [13]:
def train(inp,target,batch_size):
    hidden = decoder.init_hidden(batch_size)
    if use_gpu:
        inp = inp.cuda()
        if isinstance(hidden, tuple):
            hidden = (hidden[0].cuda(), hidden[1].cuda())
        else:
            hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0
    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

### Load Model From Previous Training and continue

In [ ]:
n_epochs = 5000
print_every = 200
plot_every = 50
hidden_size = 512
n_layers = 2
lr = 0.005
chunk_len = 100
batch_size = 50


decoder = RNN(n_characters, hidden_size, n_characters,"lstm", n_layers)
if use_gpu:
    decoder.cuda()
decoder.load_state_dict(torch.load('model.pt'))
decoder.train()

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set(chunk_len,batch_size),batch_size)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[(%d %d%%) %.4f]' % ( epoch, epoch / n_epochs * 100, loss))
        print(evaluate(decoder,'令', 50, use_gpu=use_gpu), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[(200 4%) 1.6212]
令狐冲心想：“我几时得罪了你，有个人，便练了几年，众位婆婆但先前家定逸师太一番新水，入伏圣姑。两位明鉴 



/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


In [18]:
for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set(chunk_len,batch_size),batch_size)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[(%d %d%%) %.4f]' % ( epoch, epoch / n_epochs * 100, loss))
        print(evaluate(decoder,'令', 50, use_gpu=use_gpu), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[(200 4%) 1.8789]
令狐冲迷迷糊糊的仙咬住了他，令狐冲双目向岭跌出来，说道：“你身子横斩，吓他一个道理。”
令狐冲道：“我 



/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:29: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[(400 8%) 1.7620]
令狐冲道：“方师哥，令狐兄，请你将他补上了。”
突见那时汉子叫道：“且慢。”令狐冲心中一阵凄凉，对这婆 

[(600 12%) 1.7325]
令狐冲心中一喜，忙道：“既然不是寻常各派的少年，这是田某五毒教七个方位相授，命小师妹不利于我，那不必用 

[(800 16%) 1.8344]
令狐冲笑道：“令狐冲，你对我这位丁兄包含了你们吗？”老公公道：“为何？”身子一晃，右足一抓，长剑一骨， 

[(1000 20%) 1.7577]
令狐冲大笑，都道：“左掌门是由于徒成不入，咱们便随也没有，但如今晚，也都是乐厚的贵教，我和曲大哥多谢你 

[(1200 24%) 1.7336]
令狐冲笑道：“方证大师是衡山派的，前来生火入怀。”令狐冲道：“人家此番有伤，我总是不明不白。刚才你快活 

[(1400 28%) 1.7865]
令狐冲不由得心动，摸不到余人，但片刻间有新之意，心想已是颇觉不合，便以此刻细看，但她竟始终住不清楚，正 

[(1600 32%) 1.7221]
令狐冲道：“你认得我是偷听我的传人，但从来就没放过，咱们一起死了好一会，忽然脸上阴晴不定。
令狐冲听了 

[(1800 36%) 1.8146]
令狐冲道：“小师妹，令狐冲，你死了。”将那碗馄饨的模字也不大起议。
田伯光又道：“那自然不会怪你的。我 

[(2000 40%) 1.6951]
令狐冲一个自出。”那婆婆道：“正是。”一伸手，从怀中取出一个瓷瓶，轿子后上雪了点头，默然不语，心下均有 

[(2200 44%) 1.6148]
令狐冲上了去，令狐冲长剑一开，将玉米棒喝震死了。
令狐冲大惊，叫道：“师娘！”探她头上一把，轻声道：“ 

[(2400 48%) 1.7704]
令狐冲对自己颇有疑忌之意。
仪清道：“如果你我份量不休，担心便是。”
令狐冲道：“你……你别怕！”
令 

[(2600 52%) 1.7953]
令狐冲道：“定闲、定逸两位师伯，是华山派掌门岳先生门下，剑术之精，难以自制，和你们一齐冲上了一个半，无 

[(2800 56%) 1.7054]
令狐冲又觉不答，心中却是一觉念头，听他说笑话也是在骂她。令狐冲叫道：“你真的死样？”令狐冲叹了口气，说 

[(3000 60%) 1.6441]
令狐冲脸上一阵冰冷。那姓谭的道：“六……小…

In [19]:
torch.save(decoder.state_dict(), 'model.pt')